This notebook shows how to prepare the table of coverages using bedtools.

**Prerequisites**

* **bedtools** This notebook was tested with bedtools 2.24
* A folder **alignments_all** with the BAM files indexed with a sensible name for each sample. We use the convention **Line_Library** to be able to have two or more libraries coming from the same line. The bam files must be already indexed with **samtools** 
* **exons.bed** is a bed file with the coordinate of eeach exon. It could be any other feature if it is sensible in the sample. 

**Notes**

This notebook doesn't contain example files because the size of the bam files. 


** Extracting the coverage **

The first step is to create a folder with the file with the coverage of each file. The coverageBed command can be executed in parallel on a cluster. 

In [ ]:
mkdir -p bedCovMerged_all
for f in `ls alignments_all | grep -v bai`; do
    echo $f
    filename="${f%.*}"
    if [ -f bedCovMerged_all/$filename.tab ] ; then
        echo "Already extracted"
    else
        coverageBed -a References/exons.bed -b alignments_all/$f -sorted -g References/chr_order.txt  > bedCovMerged_all/$filename.tab 
    fi
done

Go to the folder with the coverages. 

In [11]:
folder="bedCovMerged_all"
cd $folder

Extract the column with the coverage (4) and save the coverage in a separate name just with the coverage of the column with the coverage in a temp file. 

In [12]:
for fullfile in `ls Cadenza*.tab`; do
    #echo $fullfile
    filename=$(basename "$fullfile")
    extension="${filename##*.}"
    filename="${filename%.*}"
    #echo $filename
    echo $filename > cov.${filename}.txt
    awk '{print $4}' $fullfile >> cov.${filename}.txt
    
done


Prepare the coordinates of each exon in a single column. This convention is later used in the R script to get put all the exons in a single contig together.  

In [14]:
echo "" > exonNames.txt
awk '{print $1":"$2":"$3}' $fullfile >> exonNames.txt
head exonNames.txt


IWGSC_CSS_1AL_scaff_18283:1:435
IWGSC_CSS_1AL_scaff_18283:527:1204
IWGSC_CSS_1AL_scaff_19831:594:685
IWGSC_CSS_1AL_scaff_19831:803:927
IWGSC_CSS_1AL_scaff_72977:87:218
IWGSC_CSS_1AL_scaff_72977:294:374
IWGSC_CSS_1AL_scaff_84503:12:592
IWGSC_CSS_1AL_scaff_84503:14:1061
IWGSC_CSS_1AL_scaff_84503:1618:1779


Merge all the files in a single file. The ulimit is only in case the system complains of too many files open

In [15]:
ulimit -S -n 2048
paste exonNames.txt cov*.txt > allMergedCoverages.tab

paste: cov.Cadenza0608_LIB11452.txt: Too many open files
